In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)

In [ ]:
default_home_value <- 800000
default_down_payment <- default_home_value * 0.20
default_interest_rate <- 0.03
# term_yrs <- 30
start_date <- floor_date(unit='month', today()+ months(1))
start_date <- ymd(20201101)
default_payment <- 2200
yearly_costs <- 313*12
pmi <- 118
print(paste('downPayment:', default_down_payment))

In [ ]:
month_seq <- seq(start_date,start_date+years(60), by = 'months')

In [ ]:
day_seq <- seq(start_date,start_date+years(30), by = 'days')

In [ ]:
default_payment_schedule <- tibble(month_date = month_seq, 
                           standard_payment = rep(default_payment,times=length(month_seq)),
                                   payment = default_payment)

In [ ]:
override_payment_schedule <- function(input_payments, standard_payment_schedule){
    payment_schedule <- standard_payment_schedule %>%
    left_join(input_payments, by=c('month_date'='month_date')) %>%
    mutate(
    payment = ifelse(
        is.na(override_payment), 
        standard_payment, 
        override_payment))
    return(payment_schedule)
}

In [ ]:
mortgage_calc <- function(home_value, down_payment, interest_rate, payment_schedule, tag) {
    current_value = home_value - down_payment
first_payment = 0
next_value= current_value*(1+interest_rate/12)-first_payment
df <- tibble(day = start_date, 
             current_value = current_value, 
             payment = first_payment,
             interest=0,
             next_value, additional_costs = pmi+yearly_costs/12, pmi=pmi)
for (month_index in 1:length(month_seq)){
    this_payment <- payment_schedule[[month_index, 'payment']]
    this_interest <- next_value*(interest_rate/12)
    this_pmi <- ifelse(home_value*0.8 < next_value, pmi, 0)
    next_value <- next_value + this_interest - this_payment 
    temp_df<-tibble(
        day = month_seq[month_index], 
             current_value =  ifelse(next_value> 0, next_value, 0), 
             payment = ifelse(next_value> 0, this_payment, 0),
        interest = this_interest,
             next_value = ifelse(next_value> 0, next_value, 0),
        additional_costs=this_pmi+yearly_costs/12,
        pmi=this_pmi
            )
    df <- df %>% rbind(temp_df)
    
}
    result_df <- df %>% 
   filter(next_value > 0) %>%
mutate(
    cum_payment = cumsum(payment),
    cum_interest = cumsum(interest),
    cum_pmi = cumsum(pmi),
    cum_total = cum_payment + cum_pmi
) %>% 
     mutate(tag=tag) 
    return(result_df)
                             }

In [ ]:
test_mortgage_df <- mortgage_calc(default_home_value, default_down_payment, default_interest_rate, default_payment_schedule, 'standard')
testthat::expect_equal(test_mortgage_df[[nrow(test_mortgage_df),'cum_payment']],1150600)

In [ ]:
test_mortgage_df %>% head()

In [ ]:
standard_payment <- default_payment
standard_payment_schedule <- tibble(month_date = month_seq, 
                           standard_payment = rep(standard_payment,times=length(month_seq)),
                                   payment = standard_payment)

In [ ]:
override_payment <- 3000
override_payments3 <- tibble(month_date = month_seq, 
                                   override_payment = override_payment)

In [ ]:
override_payments <-
tibble(month_date = month_seq[1:5], override_payment=4224) %>%
rbind(tibble(month_date = month_seq[6:11], override_payment=4224)) %>%
rbind(tibble(month_date = month_seq[13:600], override_payment=4224)) %>%
mutate(month_date = ymd(month_date))

In [ ]:
override_payments2 <-
tibble(month_date = month_seq[1:5], override_payment=4224) %>%
rbind(tibble(month_date = month_seq[6:11], override_payment=2224)) %>%
rbind(tibble(month_date = month_seq[13:600], override_payment=4224)) %>%
mutate(month_date = ymd(month_date))

In [ ]:
override_payments2

In [ ]:
full_value <- default_home_value
down <- default_down_payment
down

In [ ]:
standard_mortgage_df <- mortgage_calc(full_value, down, default_interest_rate, standard_payment_schedule, 'standard')
override_mortgage_df <- mortgage_calc(full_value, down, default_interest_rate, override_payment_schedule(override_payments,standard_payment_schedule), 'override4224')
override_mortgage_df2 <- mortgage_calc(full_value, down, default_interest_rate, override_payment_schedule(override_payments2,standard_payment_schedule), 'override4224_minus6')
# override_mortgage_df3 <- mortgage_calc(full_value, down, default_interest_rate, override_payment_schedule(override_payments3,standard_payment_schedule), 'override3000')


larger_mortgage_df <- mortgage_calc(full_value, down+20000, default_interest_rate, override_payment_schedule(override_payments,standard_payment_schedule), 'larger_down4069')
larger_only_mortgage_df <- mortgage_calc(full_value, down+20000, default_interest_rate, standard_payment_schedule, 'larger_down_only')

In [ ]:
mortgage_df <- rbind(standard_mortgage_df,
                     override_mortgage_df,
                     override_mortgage_df2
#                      override_mortgage_df3,
#                      larger_mortgage_df,
#                      larger_only_mortgage_df
                    )
mortgage_df %>% 
group_by(tag) %>%
arrange(desc(day)) %>%
slice(1)

In [ ]:
mortgage_df %>%
reshape2::melt(id.vars=c('day', 'tag')) %>%
ggplot(aes(x=day, y= value, color=tag)) +
geom_line() +
facet_grid(variable ~. , scales = 'free')

In [ ]:
mortgage_df %>%
mutate(year_date = floor_date(unit='year', day )) %>%
select(-day) %>%
group_by(year_date, tag) %>%
summarise_all(sum) %>%
reshape2::melt(id.vars=c('year_date', 'tag')) %>%
# head()
ggplot(aes(x=year_date, y= value, color=tag)) +
geom_line() +
facet_grid(variable ~. , scales = 'free')

In [ ]:
mortgage_calc_day <- function(home_value, down_payment, interest_rate, payment_schedule, tag) {
    current_value = home_value - down_payment
first_payment = 0
next_value= current_value*(1+interest_rate/365)-first_payment
df <- tibble(day = start_date, 
             current_value = current_value, 
             payment = first_payment,
             interest=0,
             next_value, additional_costs = pmi/30+yearly_costs/365, pmi=pmi/30)
for (day_index in 1:length(day_seq)){
    if (next_value >0 ){
    days_payment <- payment_schedule[payment_schedule$month_date==day_seq[day_index], 'payment']
    if (nrow(days_payment)==1) {
        this_payment<-days_payment[[1]]
    } else {
        this_payment<-0
    }
    this_interest <- next_value*(interest_rate/365)
    this_pmi <- ifelse(home_value*0.8 < next_value, pmi, 0)
    next_value <- next_value + this_interest - this_payment 
    temp_df<-tibble(
        day = day_seq[day_index], 
             current_value =  ifelse(next_value> 0, next_value, 0), 
             payment = ifelse(next_value> 0, this_payment, 0),
        interest = this_interest,
             next_value = ifelse(next_value> 0, next_value, 0),
        additional_costs=this_pmi/30+yearly_costs/365,
        pmi=this_pmi/30
            )
    df <- df %>% rbind(temp_df)
        }
    
}
    result_df <- df %>% 
   filter(next_value > 0) %>%
mutate(
    cum_payment = cumsum(payment),
    cum_interest = cumsum(interest),
    cum_pmi = cumsum(pmi),
    cum_total = cum_payment + cum_pmi
) %>% 
     mutate(tag=tag) 
    return(result_df)
                             }

In [ ]:
default_payment_schedule[default_payment_schedule$month_date==day_seq[1],'payment'][[1]]

In [ ]:
test_mortgage_df <- mortgage_calc_day(default_home_value, default_down_payment, default_interest_rate, default_payment_schedule, 'standard')
testthat::expect_lt(abs(test_mortgage_df[[nrow(test_mortgage_df),'cum_payment']]-(427405+221299)), 5000)

In [ ]:
standard_payment <- default_payment/2
payment_seq <- seq(start_date,start_date+years(30), by = '2 weeks')
standard_payment_schedule_2_weeks <- tibble(month_date = payment_seq, 
                           standard_payment = rep(standard_payment,times=length(payment_seq)),
                                   payment = standard_payment)

In [ ]:
override_payment <- (6500-2000-431)/2
override_payments_2_weeks <- tibble(month_date = payment_seq, 
                                   override_payment = override_payment)

In [ ]:
standard_mortgage_df <- mortgage_calc(full_value, down, default_interest_rate, standard_payment_schedule, 'standard')

override_mortgage_df <- mortgage_calc(full_value, down, default_interest_rate, override_payment_schedule(override_payments,standard_payment_schedule), 'override4069')
override_mortgage_df2 <- mortgage_calc(full_value, down, default_interest_rate, override_payment_schedule(override_payments2,standard_payment_schedule), 'override4869')
override_mortgage_df3 <- mortgage_calc(full_value, down, default_interest_rate, override_payment_schedule(override_payments3,standard_payment_schedule), 'override3000')


larger_mortgage_df <- mortgage_calc(full_value, down+20000, default_interest_rate, override_payment_schedule(override_payments,standard_payment_schedule), 'larger_down4069')
larger_only_mortgage_df <- mortgage_calc(full_value, down+20000, default_interest_rate, standard_payment_schedule, 'larger_down_only')

In [ ]:
standard_2_week_mortgage_df <- mortgage_calc_day(full_value, down, default_interest_rate, standard_payment_schedule_2_weeks, 'standard_2_weeks')


In [ ]:
standard_2_week_mortgage_df %>% tail()

In [ ]:
override_2_week_mortgage_df <- mortgage_calc_day(full_value, down, default_interest_rate, override_payment_schedule(override_payments_2_weeks,standard_payment_schedule_2_weeks), 'override_4069_2_weeks')


In [ ]:
mortgage_df <- rbind(standard_mortgage_df,
                     override_mortgage_df,
                     override_mortgage_df2,
#                      override_mortgage_df3,
                     larger_mortgage_df,
                     larger_only_mortgage_df,
#                      standard_2_week_mortgage_df,
#                     override_2_week_mortgage_df
                    )
mortgage_df %>% 
group_by(tag) %>%
arrange(desc(day)) %>%
slice(1)

In [ ]:
mortgage_df %>%
reshape2::melt(id.vars=c('day', 'tag')) %>%
ggplot(aes(x=day, y= value, color=tag)) +
geom_line() +
facet_grid(variable ~. , scales = 'free')

In [ ]:
mortgage_df %>% tail()

In [ ]:
mortgage_df %>%
mutate(year_date = floor_date(unit='year', day )) %>%
select(-day) %>%
group_by(year_date, tag) %>%
summarise(
    current_value= max(current_value),
    payment = sum(payment),
    interest = sum(interest),
    additional_costs = sum(additional_costs),
    pmi = sum(pmi),
    cum_payment=max(cum_payment),
    cum_interest=max(cum_interest),
    cum_pmi=max(cum_pmi),
    cum_total=max(cum_total),
    ) %>%
reshape2::melt(id.vars=c('year_date', 'tag')) %>%
# head()
ggplot(aes(x=year_date, y= value, color=tag)) +
geom_line() +
facet_grid(variable ~. , scales = 'free')

In [ ]:
mortgage_df %>%
mutate(month_date = floor_date(unit='month', day )) %>%
select(-day) %>%
group_by(month_date, tag) %>%
summarise(
    current_value= max(current_value),
    payment = sum(payment),
    interest = sum(interest),
    additional_costs = sum(additional_costs),
    pmi = sum(pmi),
    cum_payment=max(cum_payment),
    cum_interest=max(cum_interest),
    cum_pmi=max(cum_pmi),
    cum_total=max(cum_total),
    ) %>%
reshape2::melt(id.vars=c('month_date', 'tag')) %>%
# head()
ggplot(aes(x=month_date, y= value, color=tag)) +
geom_line() +
facet_grid(variable ~. , scales = 'free')